In [1]:
import psycopg2
from sqlalchemy import create_engine
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time
import random
import plotly.express as px
from datetime import timedelta

In [2]:
general_path = 'E:/paper2_analysis/connect_flood_roads_to_trafficmodel'
figures_path = f'{general_path}/figures'
input_path = f'{general_path}/input'
output_path = f'{general_path}/output'

----------

#### result from flood

In [3]:
## json with all flood depth on sample points
name_flood_result = 'flooddepth_time_road_segments_df_dh_scen1_voorpiek_sz2000_uuid_66e0b6ee-e4af-11eb-a904-acde48001122'
flooded_roads = pd.read_json(f'{input_path}/{name_flood_result}.json')
flooded_roads.head(2)


,116555,89886,14563,71854,83429,80724,90708,81565,84918,67656,...,17558,73571,102733,74322,85635,17553,88273,17442,17202,96089
2000-01-01 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-01-01 18:05:10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# df_osmid_flood.to_csv(f'{output_path}/df_osmid_flood.csv', header= True, sep =';', decimal=',' )

In [5]:
#shapefile of all point where sample is taken from
geoms_floodcalc_nodes_name = 'created_wegdelen_OSM_centroids_cell_id_floodselection_without_bridges_and_initwater'
geoms_floodcalc_nodes = gpd.read_file(f'{input_path}/{geoms_floodcalc_nodes_name}.shp')

#change osmid to integers (instead oof float)
geoms_floodcalc_nodes = geoms_floodcalc_nodes.astype({"osmid": int})

In [6]:
len(geoms_floodcalc_nodes)

77171

#### calculate flood arrival time for every node

In [9]:
## create db with arrival times for every calculation point
df_segments_flood = pd.DataFrame(columns=['segmentid', 'arrivaltime', 'firstdepth', 'geometry'])

#get all column names
list_segment_in_comp = flooded_roads.columns.values.tolist()

df_segments_flood["segmentid"] = list_segment_in_comp
df_segments_flood.head(1)

,segmentid,arrivaltime,firstdepth,geometry
0,116555,NaN,NaN,NaN


In [10]:
## derive flood arrival and first depth--> append it to dataframe
for index, value  in flooded_roads.items():
    
    for i in range(len(value)):
                   if value[i] > 0.01:
                    arrival_time = flooded_roads.index[i]
                    first_depth = value[i]
                    osmid = value.name
                    geom = geoms_floodcalc_nodes
                    df_segments_flood.loc[df_segments_flood.segmentid == osmid, 'arrivaltime'] = arrival_time
                    df_segments_flood.loc[df_segments_flood.segmentid == osmid, 'firstdepth'] = first_depth
            
                    break

df_segments_flood.to_csv(f'{output_path}/df_segments_flood.csv', header= True, sep =';', decimal=',' )               
df_segments_flood.head()

,segmentid,arrivaltime,firstdepth,geometry
0,116555,2000-01-01 20:45:03,0.079,NaN
1,89886,2000-01-01 18:30:03,0.9117,NaN
2,14563,2000-01-02 04:25:03,0.068319,NaN
3,71854,2000-01-01 19:40:03,0.167837,NaN
4,83429,2000-01-01 18:50:03,0.259214,NaN


### join geoms of segments with new dataframe

In [17]:
df_segments_flood_geoms_temp  = pd.merge(df_segments_flood,geoms_floodcalc_nodes,left_on='segmentid', right_on='roadsegmen' )
df_segments_flood_geoms_temp["geom"] = df_segments_flood_geoms_temp.geometry_y
df_segments_flood_geoms_temp = df_segments_flood_geoms_temp.drop(columns=['geometry_y', 'geometry_x'])

df_segments_flood_geoms = gpd.GeoDataFrame(df_segments_flood_geoms_temp, geometry= 'geom')
df_segments_flood_geoms.head(3)


,segmentid,arrivaltime,firstdepth,maaiveld,functie,roadsegmen,cell_id,osmid,highway,geom
0,116555,2000-01-01 20:45:03,0.079,1.393333,rijbaan regionale weg,116555,32434.0,6642407,primary,POINT (115138.996 547717.726)
1,89886,2000-01-01 18:30:03,0.9117,0.396667,rijbaan lokale weg,89886,11933.0,6640668,tertiary,POINT (112419.708 551921.701)
2,14563,2000-01-02 04:25:03,0.068319,0.940000,rijbaan regionale weg,14563,54417.0,6645840,secondary,POINT (109090.447 541626.305)


In [19]:
df_segments_flood_geoms.arrivaltime

0      2000-01-01 20:45:03
1      2000-01-01 18:30:03
2      2000-01-02 04:25:03
3      2000-01-01 19:40:03
4      2000-01-01 18:50:03
              ...         
621    2000-01-01 21:55:03
622    2000-01-01 19:25:03
623    2000-01-01 23:55:04
624    2000-01-02 01:20:00
625    2000-01-01 22:05:03
Name: arrivaltime, Length: 626, dtype: object

In [ ]:
## arrival_time_naar_minutes_after_start

In [18]:
type(df_segments_flood_geoms)

geopandas.geodataframe.GeoDataFrame

In [ ]:
df_segments_flood_geoms.to_file(f'{output_path}/df_segments_flood_geoms.geojson')

In [ ]:

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1)
test.plot(ax=ax,alpha = 0.1, color = '#ade8f4')